In [1]:
import napari
from pathlib import Path
from skimage import io
import os
import czifile
import math
import numpy as np
from tqdm.auto import tqdm

C:\Users\Nathan\miniforge3\envs\napari\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Zeiss lattice lightsheet

In [2]:
os.listdir(Path(r'Z:\The_Holy_de_Broglies\2025-08-13 Lattice Lightsheet\Macrophages\2025-08-13\mouse'))

['middle-01.czi',
 'middle-01_MIP.czi',
 'middle-01_processed.czi',
 'middle-01_processed.ome.tiff',
 'middle-02.czi',
 'middle-02_DCVSettingsTest.czi',
 'middle-02_DCVSettingsTest.czt',
 'middle-02_MIP.czi',
 'middle-02_processed.czi',
 'middle-03.czi',
 'middle-03_DCVSettingsTest.czi',
 'middle-03_DCVSettingsTest.czt',
 'middle-03_MIP.czi',
 'middle-03_processed.czi',
 'middle-04.czi',
 'middle-04_MIP.czi',
 'middle-04_processed.czi',
 'middle-05.czi',
 'middle-05_MIP.czi',
 'middle-05_processed.czi',
 'middle-06.czi',
 'middle-06_MIP.czi',
 'middle-06_processed.czi',
 'middle-11-Lattice Lightsheet-01.czi',
 'middle-11.czi',
 'middle-11_MIP.czi',
 'middle-15-Lattice Lightsheet-02.czi',
 'middle-15.czi',
 'middle-15_MIP.czi',
 'wildtype-01-deskewed-deconv.czi',
 'wildtype-01-deskewed.czi',
 'wildtype-01.czi',
 'wildtype-01.ome.tiff',
 'wildtype-01_MIP.czi',
 'wildtype-01_MIP.ome.tiff',
 'wildtype-01_processed-01.czi',
 'wildtype-01_processed.czi']

In [3]:
viewer = napari.Viewer(title = 'blender io')

### Mouse macrophages

In [4]:
image_path = Path(r'Z:\The_Holy_de_Broglies\2025-08-13 Lattice Lightsheet\Macrophages\2025-08-13\mouse\wildtype-01-deskewed-deconv.czi')

In [5]:
%%time 

image = np.squeeze(czifile.imread(image_path))

CPU times: total: 1min 46s
Wall time: 2min 4s


In [6]:
image.shape

(2, 114, 7576, 6407)

### Deskew (no longer necessary)

In [49]:
from scipy.ndimage import shift
from tqdm.auto import tqdm
import numpy as np

In [56]:
C, Z, Y, X = image.shape

angle_deg = 31.8
xy_um = 0.145
z_um  = 0.400
px_shift_per_z = (z_um / xy_um) * np.tan(np.deg2rad(angle_deg))

max_shift = int(np.ceil(abs((Z - 1) * px_shift_per_z)))
X_expanded = X + max_shift
deskewed = np.zeros((C, Z, Y, X_expanded), dtype=image.dtype)

x0 = max_shift // 2
sign = +1.0  # positive sign as requested

for c in range(C):
    for z in tqdm(range(Z), leave=False, desc=f"Deskew C{c}"):
        x_pos = x0 + sign * z * px_shift_per_z
        x_int = int(np.floor(x_pos))
        frac  = x_pos - x_int

        x_start = max(0, min(X_expanded - X, x_int))
        canvas = np.zeros((Y, X_expanded), dtype=image.dtype)
        canvas[:, x_start : x_start + X] = image[c, z]

        if abs(frac) > 1e-6:
            canvas = shift(canvas, shift=(0, frac), order=1, mode="nearest", prefilter=False)

        deskewed[c, z] = canvas

Deskew C0:   0%|          | 0/2251 [00:00<?, ?it/s]

Deskew C1:   0%|          | 0/2251 [00:00<?, ?it/s]

## Visualise

In [10]:
viewer.add_image(image, channel_axis=0, name = 'wildtype deskewed')

[<Image layer 'wildtype deskewed' at 0x20da3e80ad0>,
 <Image layer 'wildtype deskewed [1]' at 0x20da3d65b10>]

In [12]:
viewer.layers

[<Image layer 'wildtype deskewed reshaped' at 0x1de861225d0>, <Image layer 'wildtype deskewed reshaped [1]' at 0x1def787f150>]

In [14]:
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'

In [15]:
for i in [0,1]:
    print(viewer.layers[i].contrast_limits)

[154.66696914700543, 857.9282084521649]
[300.0, 800.0]
Rendering frames...


100%|██████████████████████████████████████████████████████| 91/91 [00:04<00:00, 19.08it/s]


Rendering frames...


  2%|█▏                                                     | 2/91 [00:00<00:07, 11.97it/s]


In [16]:
print()


Rendering frames...


  0%|                                                      | 1/601 [00:00<01:30,  6.65it/s]


In [17]:
print()


Rendering frames...


100%|████████████████████████████████████████████████████| 601/601 [00:29<00:00, 20.60it/s]


Rendering frames...


100%|████████████████████████████████████████████████████| 601/601 [00:28<00:00, 20.87it/s]


In [13]:
for i in [0,1]:
    viewer.layers[i].scale = [0.4,0.145,0.145]

### Scape

In [5]:
os.listdir(Path(r'Y:\The_Holy_de_Broglies\ASI SCAPE\20250813\Processed'))

['20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome.deskewed_TRUE_Pos00_CH00_T000.tif',
 'MAX20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome_deskewed_TRUE_Pos00_CH00_T000.tif',
 'Thumbs.db']

In [6]:
viewer = napari.Viewer(title = 'scape')

### Mouse macrophages

In [7]:
image_path = Path(r'Y:\The_Holy_de_Broglies\ASI SCAPE\20250813\Processed\20250813_macrophages_GLA3488_multiPos_lowerLaserP_MMStack_Pos0.ome.deskewed_TRUE_Pos00_CH00_T000.tif')

In [9]:
%%time 

image = io.imread(image_path)

CPU times: total: 7.48 s
Wall time: 25.5 s


In [10]:
image.shape

(3, 81, 15, 1321, 1005)

### Separate positions and tile into pseudo config

In [16]:
# Input: image shaped (C, Z, P, Y, X) e.g. (3, 81, 15, 1321, 1005)
C, Z, P, Y, X = image.shape

# Choose a compact grid to tile the P=15 positions (row-major, arbitrary order)
cols = math.ceil(math.sqrt(P))
rows = math.ceil(P / cols)

# Allocate output mosaic: (C, Z, Y_total, X_total)
mosaic = np.zeros((C, Z, rows * Y, cols * X), dtype=image.dtype)

# Tile positions into the mosaic
p = 0
for r in tqdm(range(rows)):
    for c in tqdm(range(cols)):
        if p >= P:
            break
        ys, ye = r * Y, (r + 1) * Y
        xs, xe = c * X, (c + 1) * X
        mosaic[:, :, ys:ye, xs:xe] = image[:, :, p, :, :]
        p += 1

# Result: 'mosaic' is (C, Z, Y_total, X_total)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [49]:
from scipy.ndimage import shift
from tqdm.auto import tqdm
import numpy as np

In [56]:
C, Z, Y, X = image.shape

angle_deg = 31.8
xy_um = 0.145
z_um  = 0.400
px_shift_per_z = (z_um / xy_um) * np.tan(np.deg2rad(angle_deg))

max_shift = int(np.ceil(abs((Z - 1) * px_shift_per_z)))
X_expanded = X + max_shift
deskewed = np.zeros((C, Z, Y, X_expanded), dtype=image.dtype)

x0 = max_shift // 2
sign = +1.0  # positive sign as requested

for c in range(C):
    for z in tqdm(range(Z), leave=False, desc=f"Deskew C{c}"):
        x_pos = x0 + sign * z * px_shift_per_z
        x_int = int(np.floor(x_pos))
        frac  = x_pos - x_int

        x_start = max(0, min(X_expanded - X, x_int))
        canvas = np.zeros((Y, X_expanded), dtype=image.dtype)
        canvas[:, x_start : x_start + X] = image[c, z]

        if abs(frac) > 1e-6:
            canvas = shift(canvas, shift=(0, frac), order=1, mode="nearest", prefilter=False)

        deskewed[c, z] = canvas

Deskew C0:   0%|          | 0/2251 [00:00<?, ?it/s]

Deskew C1:   0%|          | 0/2251 [00:00<?, ?it/s]

In [17]:
mosaic.shape

(3, 81, 5284, 4020)

In [23]:
mosaic = np.max(mosaic, axis=0)

## Visualise

In [24]:
viewer.add_image(mosaic, name = 'mosaic')

<Image layer 'mosaic' at 0x1be06a59b90>

In [25]:
viewer.layers

[<Image layer 'mosaic' at 0x1be06a59b90>]

In [27]:
for i in viewer.layers:
    i.scale = [0.5, 0.391, 0.391]

In [28]:
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'μm'

In [31]:
viewer.scale_bar.font_size = 20
viewer.scale_bar.ticks = False

Rendering frames...


100%|████████████████████████████████████████| 151/151 [00:05<00:00, 26.56it/s]


Rendering frames...


100%|████████████████████████████████████████| 151/151 [00:05<00:00, 26.98it/s]


In [29]:
for i in [0,1]:
    print(viewer.layers[i].contrast_limits)

[0.0, 117.0]


IndexError: list index out of range

### Save out Zeiss mouse image as stack of planes

In [3]:
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import imageio.v3 as iio
import czifile  # or tifffile if you’ve exported to TIFF already

# ---------- CONFIG ----------
out_root = Path(r"C:\Users\Nathan\Desktop\data\LLS7_acq_as_planes")    # output base folder
robust_percentiles = (1.0, 99.8)                               # intensity scaling per channel
overwrite = True                                              # guard against accidental reruns
# ----------------------------
image_path = Path(r'Z:\The_Holy_de_Broglies\2025-08-13 Lattice Lightsheet\Macrophages\2025-08-13\mouse\wildtype-01-deskewed-deconv.czi')
out_root.mkdir(parents=True, exist_ok=True)

C:\Users\Nathan\miniforge3\envs\napari\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

# Load and squeeze to (C, Z, Y, X). Your example was (2, 114, 7576, 6407).
vol = np.squeeze(czifile.imread(image_path))
if vol.ndim != 4:
    raise ValueError(f"Expected (C, Z, Y, X); got {vol.shape}")

C, Z, Y, X = vol.shape
print(f"Loaded volume: C={C} Z={Z} Y={Y} X={X}")


Loaded volume: C=2 Z=114 Y=7576 X=6407


In [5]:
vol.dtype

dtype('uint16')

In [6]:
size_gb = vol.nbytes / (1024**3)
print(f"Volume size: {size_gb:.2f} GB")

Volume size: 20.61 GB


In [7]:
overwrite = False
percentile_subsample = 4               # >=1. Use 1 for full-res percentiles; >1 strides for speed
# ----------------------------------------

bitdepth = 16

# Choose dtype conversion
if bitdepth == 16:
    out_dtype = np.uint16
    maxv = np.iinfo(out_dtype).max
elif bitdepth == 8:
    out_dtype = np.uint8
    maxv = np.iinfo(out_dtype).max
else:
    raise ValueError("bitdepth must be 8 or 16")

print('maxv determined, outputting beginning')


for c in tqdm(range(C), desc = 'iterating over channels'):
    ch_dir = out_root / f"channel_{c:02d}"
    if ch_dir.exists() and not overwrite:
        print(f"Skipping existing channel dir: {ch_dir} (set overwrite=True to replace)")
        continue
    ch_dir.mkdir(parents=True, exist_ok=True)

    # ---- LOW-MEM PERCENTILES (channel stays uint16) ----
    # Compute robust window on a strided subsample to save time/RAM.
    ch_u16 = vol[c]  # (Z,Y,X), dtype likely uint16
    if percentile_subsample > 1:
        s = percentile_subsample
        sample = ch_u16[::max(1, Z // min(Z, Z)), ::s, ::s]  # stride in YX only
        p_low, p_high = np.percentile(sample, robust_percentiles)
    else:
        p_low, p_high = np.percentile(ch_u16, robust_percentiles)

    scale_den = float(max(p_high - p_low, 1e-6))

    # ---- SLICE-BY-SLICE NORMALIZE & WRITE (no big float array) ----
    for z in tqdm(range(Z), desc=f"Channel {c}: writing"):
        # Promote *just this slice* to float32 for math, then drop it.
        sl = ch_u16[z].astype(np.float32, copy=False)
        sl = (sl - p_low) / scale_den
        sl = np.clip(sl, 0.0, 1.0)

        # Quantize to output bitdepth
        plane = (sl * maxv + 0.5).astype(out_dtype, copy=False)
        iio.imwrite(ch_dir / f"z_{z:04d}.png", plane, extension=".png")

print(f"Done. Import in Blender via: File → Import → Images as Planes.")
print(f"Import one channel folder at a time; enable 'Animate Image Sequence' if desired.")

maxv determined, outputting beginning


iterating over channels: 100%|██████████████████████████████████████████████████████████| 2/2 [29:54<00:00, 897.17s/it]

Done. Import in Blender via: File → Import → Images as Planes.
Import one channel folder at a time; enable 'Animate Image Sequence' if desired.


In [24]:
import napari
viewer = napari.Viewer(title = 'testing blender output')
viewer.add_image(vol, channel_axis = 0)

[<Image layer 'Image' at 0x1d0229b6590>,
 <Image layer 'Image [1]' at 0x1d048429e90>]

In [20]:
[viewer.layers[i.name].contrast_limits for i in viewer.layers]

[[901.3891891891893, 1134.7585585585584], [0.0, 757.9670818505338]]

In [25]:
for i,j in zip(viewer.layers, [[901.3891891891893, 1134.7585585585584], [0.0, 757.9670818505338]]):
    i.contrast_limits = j

In [26]:
# ---------------- CONFIG ----------------
out_dir = Path(r"C:\Users\Nathan\Desktop\data\LLS7_acq_as_planes\napari_screenshots")
overwrite = False
viewer_title = "testing blender output"
# ----------------------------------------

out_dir.mkdir(parents=True, exist_ok=True)

# # Assume `vol` is already loaded: shape (C,Z,Y,X) or (Z,Y,X)
# if vol.ndim == 4:
#     viewer = napari.Viewer(title=viewer_title)
#     layer = viewer.add_image(vol, channel_axis=0, name=[f"ch{i}" for i in range(vol.shape[0])])
# elif vol.ndim == 3:
#     viewer = napari.Viewer(title=viewer_title)
#     layer = viewer.add_image(vol, name="image")
# else:
#     raise ValueError(f"Unexpected shape {vol.shape}")

Z = vol.shape[-3]  # third dimension from the end (Z axis)

for z in tqdm(range(Z), desc="Saving screenshots"):
    viewer.dims.set_point(0, z)  # step through Z
    fname = out_dir / f"z_{z:04d}.png"
    if fname.exists() and not overwrite:
        continue
    screenshot = viewer.screenshot(canvas_only=True)
    # screenshot is a numpy array (Y,X,RGBA) uint8
    from imageio.v3 import imwrite
    imwrite(fname, screenshot)

print(f"Saved {Z} screenshots to {out_dir}")

Saving screenshots: 100%|███████████████████████████████████████████████████████████| 114/114 [00:01<00:00, 101.97it/s]

Saved 114 screenshots to C:\Users\Nathan\Desktop\data\LLS7_acq_as_planes\napari_screenshots


In [28]:
viewer.sc

range(30, 60)

ERROR! Session/line number was not unique in database. History logging moved to new session 42


In [29]:
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import imageio.v3 as iio

# ---------------- CONFIG ----------------
# vol: numpy array shaped (C, Z, Y, X) or (Z, Y, X) already in memory
# Example assumes (C, Z, Y, X)
assert vol.ndim in (3, 4), f"Unexpected shape {vol.shape}"

out_dir = Path(r"C:\Users\Nathan\Desktop\data\LLS7_acq_as_planes\3rdtimesacharm")
out_dir.mkdir(parents=True, exist_ok=True)

# Contrast definition per channel:
# - If you know Napari's contrast_limits for each channel, put them here (list of (low, high)).
# - Otherwise, set to None and we'll compute robust percentiles.
napari_clims = [(901.3891891891893, 1134.7585585585584), (0.0, 757.9670818505338)]                  # e.g. [(120, 2500), (80, 1800)]
robust_percentiles = (1.0, 99.8)      # used if napari_clims is None
percentile_stride = 4                 # speedup for percentile calc (stride in YX)

gamma_per_channel = [1,1]              # e.g. [1.0, 0.8] (optional)
# Color per channel for composite export (RGB in 0–1). Common LSFM: green & magenta.
rgb_per_channel = [(0.0, 1.0, 0.0), (1.0, 0.0, 1.0)]  # adjust to your dyes
# Export modes:
export_composite_rgb = True           # 1 PNG per z, channels blended into RGB
export_per_channel_gray = False       # also save per-channel 8-bit contrasted planes
bitdepth_gray = 16                    # if per-channel, choose 8 or 16
overwrite = False
# ----------------------------------------

# ---- normalize helpers ----
def get_clims(ch_u16, idx):
    if napari_clims is not None:
        lo, hi = napari_clims[idx]
        return float(lo), float(hi)
    # robust percentiles on a strided sample (low-mem)
    s = percentile_stride
    sample = ch_u16[::max(1, ch_u16.shape[0] // ch_u16.shape[0]), ::s, ::s]
    p_low, p_high = np.percentile(sample, robust_percentiles)
    if p_high <= p_low:               # safety
        p_high = p_low + 1.0
    return float(p_low), float(p_high)

def normalize_slice(sl_u16, lo, hi, gamma=1.0):
    x = (sl_u16.astype(np.float32) - lo) / max(hi - lo, 1e-6)
    x = np.clip(x, 0.0, 1.0)
    if gamma != 1.0:
        x = x ** (1.0 / gamma)        # display-style gamma
    return x

# ---- arrange input ----
if vol.ndim == 3:
    vol = vol[None, ...]              # treat as single-channel (1, Z, Y, X)

C, Z, Y, X = vol.shape
if gamma_per_channel is None:
    gamma_per_channel = [1.0] * C
if len(rgb_per_channel) < C:
    # repeat last color if fewer specified
    rgb_per_channel += [rgb_per_channel[-1]] * (C - len(rgb_per_channel))

# ---- precompute clims per channel ----
clims = [get_clims(vol[c], c) for c in range(C)]

# ---- export ----
if export_per_channel_gray:
    if bitdepth_gray == 16:
        gray_dtype, gray_max = np.uint16, np.iinfo(np.uint16).max
    elif bitdepth_gray == 8:
        gray_dtype, gray_max = np.uint8, np.iinfo(np.uint8).max
    else:
        raise ValueError("bitdepth_gray must be 8 or 16")

for z in tqdm(range(30,60), desc="Export full-res"):
    # composite buffer (float32 0–1)
    if export_composite_rgb:
        comp = np.zeros((Y, X, 3), dtype=np.float32)

    for c in range(C):
        sl = vol[c, z]                       # uint16 slice (Y, X)
        lo, hi = clims[c]
        g = gamma_per_channel[c]
        nrm = normalize_slice(sl, lo, hi, gamma=g)  # float32 0–1

        if export_per_channel_gray:
            plane = (nrm * gray_max + 0.5).astype(gray_dtype, copy=False)
            (out_dir / f"ch{c:02d}").mkdir(exist_ok=True)
            iio.imwrite(out_dir / f"ch{c:02d}" / f"z_{z:04d}.png", plane, extension=".png")

        if export_composite_rgb:
            r, g_, b = rgb_per_channel[c]
            # additive blend (clipped); suitable for fluorescence overlays
            comp[..., 0] += nrm * r
            comp[..., 1] += nrm * g_
            comp[..., 2] += nrm * b

    if export_composite_rgb:
        comp = np.clip(comp, 0.0, 1.0)
        comp8 = (comp * 255.0 + 0.5).astype(np.uint8)
        iio.imwrite(out_dir / f"rgb_{z:04d}.png", comp8, extension=".png")

Export full-res: 100%|█████████████████████████████████████████████████████████████████| 30/30 [06:17<00:00, 12.57s/it]
